In [1]:
import sys
import csv
import requests
import re
from bs4 import BeautifulSoup

In [2]:
def load_page(url):
    with requests.get(url) as f:
        page = f.text
    return page

def get_element_text(element):
    try:
        return element.text.strip()
    except AttributeError as e:                     
        print('Element not found, error: {}'.format(e), file=sys.stderr)
        return ''

In [3]:
def get_motion_info(url):
    motion_page = BeautifulSoup(load_page(url), 'lxml')                  
    interesting_html = motion_page.find(class_='wpb_wrapper')
    titel = get_element_text(interesting_html.find('h1'))
    jaar = get_element_text(interesting_html.find('p'))[-4:]
    if ',' in jaar:
        jaar = '2'+ jaar.replace(',', '')
    elif ')' in jaar:
        jaar = '2'+ jaar.replace(')', '')
    elif jaar == ' dat':
        jaar = jaar.replace(' dat', '0')
    consdat = get_element_text(interesting_html.find_all('ul')[0]).replace(';', '; ')
    ovwdat = get_element_text(interesting_html.find_all('ul')[1]).replace(';', '; ')
    sprdat = get_element_text(interesting_html.find_all('ul')[2]).replace(';', '; ')
    return {'titel': titel, 'jaar' : jaar, 'constaterende dat' : consdat, 'overwegende dat' : ovwdat, 'spreekt uit dat' : sprdat}
    return jaar

In [4]:
def get_motions(url):
    index_page = BeautifulSoup(load_page(url), 'lxml')        
    items = index_page.find(class_="bootstrap-brendly")               
    if not items:                                             
        print('Something went wrong!', file=sys.stderr)
        sys.exit()
    data = []
    for count, row in enumerate(items.find_all(class_="col-md-4")):          
        motion_title = row.find(class_="title").text.strip()
        link = row.find(class_='download-programma').get('href')
        data.append({    
                         'id' : 'PM' + str(count + 1),
                         'motie titel': motion_title,
                         'link': link,
                        })
    return data
#get_motions('https://jongedemocraten.nl/moties/')

In [5]:
index_url = 'https://jongedemocraten.nl/moties/' 
motion_data = get_motions(index_url)                      
for row in motion_data:
    print('Scraping info on {}.'.format(row['motie titel'])) 
    url = row['link']
    motion_info = get_motion_info(url)                    
    for key, value in motion_info.items():
        row[key] = value                              

Scraping info on Verbod Cryptovaluta.
Scraping info on Gratis Schoollunch.
Scraping info on Beschikbaarheid PrEP.
Scraping info on NAVO-norm en ontwikkelingssamenwerking in wetgeving.
Scraping info on Jachttrofeeën.
Scraping info on Uitstootbelasting Etenswaren.
Scraping info on Loss and damages Klimaatfonds.
Scraping info on Duidelijkheid Websites EU.
Scraping info on Israël-Palestina.
Scraping info on Europese Omroep.
Scraping info on Genderneutraal recht.
Scraping info on Vrijwillige ouderbijdrage.
Scraping info on integriteitsbeleid (oud)bewindspersonen.
Scraping info on Bezwaarprocedures duurzaamheidsprojecten.
Scraping info on Staatssteun Oeganda.
Scraping info on Nareisbeperking families statushouders.
Scraping info on Europees Zoönosebeleid.
Scraping info on Flitshandel.
Scraping info on Korting op vleesproducten.
Scraping info on Wederopbouw Oekraïne.
Scraping info on spreidingswet en tijdspad opvanglocaties.
Scraping info on Spreidingswet en belonen doorstroom.
Scraping info 

IndexError: list index out of range

In [6]:
with open('JDmoties.csv', 'w', encoding='utf-8') as f:       
    fieldnames=['id','motie titel', 'jaar', 'constaterende dat', 'overwegende dat', 'spreekt uit dat']
    writer = csv.DictWriter(f,
                            delimiter=',',                
                            quotechar='"',                
                            quoting=csv.QUOTE_NONNUMERIC, 
                            fieldnames=fieldnames
                            )
    writer.writeheader()                                  
    for row in motion_data:
        writer.writerow({k:v for k,v in row.items() if k in fieldnames})

In [8]:
import pandas as pd

df = pd.read_csv('JDmoties.csv') 
#df['jaar'] = df['jaar'].astype(int)
df


,id,motie titel,jaar,constaterende dat,overwegende dat,spreekt uit dat
0,PM1,Verbod Cryptovaluta,dat:,Cryptovaluta geen enkele intrinsieke waarde he...,Er veel minder kwetsbare jongeren hun geld kwi...,"Bezit, handel en productie van cryptovaluta ve..."
1,PM2,Gratis Schoollunch,dat:,Steeds meer kinderen geen lunchpakket mee krij...,De schoollunches de leerprestaties van leerlin...,Alle scholen in het primair- en voortgezet ond...
2,PM3,Beschikbaarheid PrEP,dat:,De HIV-preventiepil PrEP in Nederland in een p...,De JD al van mening is dat PrEP-pillen gratis ...,De Jonge Democraten voorstander zijn van duurz...
3,PM4,NAVO-norm en ontwikkelingssamenwerking in wetg...,dat:,er in NAVO verband is afgesproken is om minste...,in tijden van bezuiniging er snel wordt bezuin...,de Nederlandse staat zowel de NAVO-norm als ee...
4,PM5,Jachttrofeeën,dat:,Er een groeiende bezorgdheid bestaat over de i...,De jacht op bedreigde diersoorten een signific...,Er een importverbod moet komen op jachttrofeeë...
...,...,...,...,...,...,...
469,PM470,Troonrede,NaN,NaN,NaN,NaN
470,PM471,Vreemdelingendetentie II,NaN,NaN,NaN,NaN
471,PM472,Vreemdelingendetentie I,NaN,NaN,NaN,NaN
472,PM473,Afhandeling gratieverzoeken,NaN,NaN,NaN,NaN


In [9]:
df['jaar'].value_counts()

2023    31
dat:    10
2022     1
Name: jaar, dtype: int64